In [ ]:
#parameters
ncbi_input_directory = "../data/taxon_10239.genbank"
output_fasta = "../data/taxon_10239.genbank/joint.fna"

In [ ]:
import ProgressMeter
import FASTX

In [ ]:
data_directory_contents = readdir(joinpath(ncbi_input_directory, "ncbi_dataset", "data"), join=true);

In [ ]:
fasta_list = String[]
ProgressMeter.@showprogress for item in data_directory_contents
    if !isdir(item)
        continue
    end
    for x in readdir(item, join=true)
        # why are other things not of interest?
        is_fasta_of_interest = occursin(basename(item), basename(x)) && occursin(r"_genomic\.fna$", x)
        if is_fasta_of_interest
            push!(fasta_list, x)
        end
    end
end
fasta_list

In [ ]:
fastx_id_set = Set()
open(output_fasta, "w") do io
    fastx_writer = FASTX.FASTA.Writer(io)
    ProgressMeter.@showprogress for fasta in fasta_list
        for fastx_record in FASTX.FASTA.Reader(open(fasta))
            id = FASTX.identifier(fastx_record)
            if !(id in fastx_id_set)
                write(fastx_writer, fastx_record)
                push!(fastx_id_set, id)
            end
        end
    end
    close(fastx_writer)
end

In [ ]:
length(fastx_id_set)